In [1]:
import torch
import timm
import numpy as np
import cv2
import matplotlib.pyplot as plt
from torchvision import transforms
from torchcam.methods import SmoothGradCAMpp
from PIL import Image


In [2]:
# Configuração do dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# Definir caminho do modelo treinado
MODEL_PATH = "model/xception_deepfake.pth"
# Diretórios com as imagens
TEST_DIR = "teste/CELEB-DF/faces"

In [4]:
# Carregar o modelo treinado
model = timm.create_model("xception41.tf_in1k", pretrained=False, num_classes=2)
model.load_state_dict(torch.load(MODEL_PATH, map_location=device))
model.to(device)
model.eval()

XceptionAligned(
  (stem): Sequential(
    (0): ConvNormAct(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNormAct2d(
        32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): ReLU(inplace=True)
      )
    )
    (1): ConvNormAct(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNormAct2d(
        64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
        (drop): Identity()
        (act): ReLU(inplace=True)
      )
    )
  )
  (blocks): Sequential(
    (0): XceptionModule(
      (shortcut): ConvNormAct(
        (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
        (bn): BatchNormAct2d(
          128, eps=0.001, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): Identity()
        )
      )
      (stack): Sequential(
     

In [5]:
# 📌 Criar transformações para normalização
transform = transforms.Compose([
    transforms.Resize((299, 299)),  # Dimensão da entrada do Xception
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

In [9]:
# 📌 Função para aplicar o Grad-CAM++ e sobrepor o heatmap
def apply_colormap_on_image(image, heatmap, alpha=0.5, colormap=cv2.COLORMAP_JET):
    """Aplica um colormap ao heatmap e mescla com a imagem original."""
    
    # Remover valores NaN e inf
    heatmap = np.nan_to_num(heatmap)

    # Normalizar o heatmap entre [0,255]
    heatmap = np.uint8(255 * (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min()))

    # Normalizar e converter para uint8 (necessário para OpenCV)
    heatmap = np.nan_to_num(heatmap)  # Remover NaN e Inf
    heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min())  # Normalizar entre 0 e 1
    heatmap = np.uint8(255 * heatmap)  # Converter para escala de 0-255 (uint8)

    # Converter para 3 canais se necessário
    if len(heatmap.shape) == 2:  # Garantir que é CV_8UC1
        heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    elif heatmap.shape[2] == 3:  # Se já for CV_8UC3, não precisa aplicar o colormap
        pass
    else:
        raise ValueError("Formato inválido de heatmap para colormap")

    # Converter para RGB (OpenCV usa BGR)
    heatmap = cv2.cvtColor(heatmap, cv2.COLOR_BGR2RGB)

    # Mesclar heatmap com a imagem original
    superimposed_img = np.uint8(heatmap * alpha + np.array(image) * (1 - alpha))
    return Image.fromarray(superimposed_img)


In [7]:
# 📌 Função para aplicar Grad-CAM++ em uma imagem
def apply_gradcam(image_path):
    # Carregar imagem e converter para RGB
    img = Image.open(image_path).convert("RGB")

    # Transformar imagem
    input_tensor = transform(img).unsqueeze(0).to(device)
    input_tensor.requires_grad_()

    # 📌 Temporariamente ativar `train()` para permitir gradientes
    model.train()

    # 📌 Aplicar Grad-CAM++
    cam_extractor = SmoothGradCAMpp(model)
    
    # Obter saída do modelo e calcular ativação Grad-CAM
    output = model(input_tensor)
    class_idx = output.argmax().item()
    activation_map = cam_extractor(class_idx, output)

    # Converter imagem para array NumPy
    img_array = np.array(img)

    # Redimensionar o mapa de ativação para o tamanho da imagem original
    heatmap = cv2.resize(activation_map[0].cpu().numpy(), (img_array.shape[1], img_array.shape[0]))

    # 📌 Aplicar o heatmap sobre a imagem original
    result = apply_colormap_on_image(img, heatmap, alpha=0.5)

    # 📌 Exibir imagem original e heatmap
    fig, ax = plt.subplots(1, 2, figsize=(12, 5))

    ax[0].imshow(img)
    ax[0].set_title("Imagem Original")
    ax[0].axis("off")

    ax[1].imshow(result)
    ax[1].set_title(f"Grad-CAM - Classe {class_idx}")
    ax[1].axis("off")

    plt.show()

    # Retornar modelo para modo de avaliação
    model.eval()

In [10]:
full_name = "videos-teste/CELEB-DF/faces/Deepfake/id0_id26_0005/frame_105_face_1.jpg"

# 📌 Testar com uma imagem de face extraída
apply_gradcam(full_name)

RuntimeError: cannot register a hook on a tensor that doesn't require gradient